# Description
作用是创建一个应变空间
相比于之前的应变空间，我们希望新的应变空间能够满足以下条件：
1、单轴拉伸 单轴压缩 双轴拉伸 双轴压缩
2、较大范围的空间 暂取[-0.05 0.045]

这样处理是为了在态密度上做出足够的区分度，检验我们的猜测，应变改变态密度，态密度决定之后的成键状态，决定吸附能。


# 0. Function part

In [1]:
import os
import re
import math
import lmdb
import torch
import random
import pickle
import shutil
import itertools
import collections
import numpy as np
import matplotlib.pyplot as plt

from ase.atoms import Atoms
# from vaspvis import standard
from ase.build import add_vacuum
from ase.calculators.emt import EMT
from ase.optimize import LBFGS, BFGS
from ase.visualize.plot import plot_atoms
from ase.io import write, read, lammpsdata
from ocpmodels.datasets import LmdbDataset
from ocpmodels.preprocessing import AtomsToGraphs
from ase.constraints import FixAtoms, FixCartesian
from ase.build import add_adsorbate, molecule, fcc111

In [2]:
def cal_strain(ini_atoms, pre_atoms, isprint=True):
    # 函数用于计算应变，变形前模型：ini_atoms, 变形后模型：pre_atoms，两个模型的属性是 ase.atoms.Atoms，
    # 如果两个模型不是Atoms(Type Error)，或者不具有应变变换(Value Error)，会提示错误。

    isAtoms = isinstance(ini_atoms, Atoms) + isinstance(pre_atoms, Atoms)
    len_queal = len(ini_atoms.positions) == len(pre_atoms.positions)
    if isAtoms * len_queal == 0:
        print("Two model are Atoms:", isAtoms == 2)
        print("Two models with equal atomic numbers :", len_queal == 1)
        raise TypeError("Model should be Atoms")
    ini_cor = ini_atoms.cell.array
    pre_cor = pre_atoms.cell.array
    # 计算形变梯度张量
    F = np.matmul(pre_cor, np.linalg.inv(ini_cor))
    E = 1 / 2 * (F.T + F) - np.identity(3)
    if isprint:
        print("strain: \n", E)
    return E

In [3]:
def opt_strain(bulk, strain, iscal=True):
    from lammps import lammps

    # 在bulk上施加应变
    strain_bulk = bulk.copy()
    strain_real = 0
    # 获取当前的晶格矩阵, 复制初始无应变构型
    cell = np.array(strain_bulk.get_cell())
    nostrain = np.identity(3)  # 单位矩阵
    # 在 x 方向上添加应变
    F = nostrain + np.array(
        [[strain[0], strain[2], 0], [strain[2], strain[1], 0], [0, 0, 0]]
    )
    newcell = np.matmul(F, cell)
    # 将新的晶格矩阵赋值给 原始 Cu 对象
    strain_bulk.set_cell(newcell, scale_atoms=True)
    # scale_Atoms=True must be set to True to ensure that ...
    # ...the atomic coordinates adapt to changes in the lattice matrix
    if iscal:
        strain_real = extracted_from_opt_strain(
            strain_bulk, lammps, cell, bulk
        )
    else:
        strain_real = cal_strain(bulk, strain_bulk)
    return (strain_bulk, strain_real)


def extracted_from_opt_strain(strain_bulk, lammps, cell, bulk):
    # 施加应变后的模型，lammps可读文件
    write("strain.lmp", strain_bulk, format="lammps-data")  # type: ignore
    # 执行lammps优化，固定了x和y的自由度，只放松了z方向的自由度
    infile = "in.strain.in"
    lmp = lammps()
    lmp.file(infile)
    atoms = lammpsdata.read_lammps_data("opt_strain.data", style="atomic")

    new_cell = atoms.get_cell()
    dot_cell = np.dot(cell[0], cell[1])
    dot_new = np.dot(new_cell[0], new_cell[1])
    if dot_cell * dot_new < 0:  # 与基础的基矢量构型不同
        new_cell[1] = new_cell[1] + new_cell[0]

    strain_bulk.set_cell(new_cell, scale_atoms=True)
    return cal_strain(bulk, strain_bulk)

In [4]:
def cal_LBFGC(ini_model, potential=EMT, fmax=1e-6, steps=1e3):
    # 执行动力学过程，默认的势函数是EMT，力收敛判断值1E-6，最大动力学步数1E3
    # 这个优化似乎不能放缩盒子
    ini_model.set_calculator(potential())  # setting the calculated potential
    # 创建 LBFGS 实例
    dyn = LBFGS(ini_model, logfile="None")
    # 进行能量最小化优化计算
    dyn.run(fmax, steps)
    # 输出优化后的结构信息和能量值
    opt_config = dyn.atoms  # initial model
    opt_energy = dyn.atoms.get_potential_energy()
    return (opt_config, opt_energy)

In [5]:
def set_strain(ini_model, strain=None, is_opt=True):
    if strain is None:
        strain = [0, 0, 0]
    # strain 表面应变由三个值控制 [ε1 ε2 ε6]
    isAtoms = isinstance(ini_model, Atoms)
    if isAtoms == 0:
        raise TypeError("Model should be Atoms")

    strain_slab = ini_model.copy()
    # 获取当前的晶格矩阵, 复制初始无应变构型
    cell = strain_slab.get_cell()

    strains = np.array([[strain[0], strain[2]], [strain[2], strain[1]]])
    deform = strains + np.identity(2)
    cell[:2, :2] = np.dot(cell[:2, :2], deform)
    # 将新的晶格矩阵赋值给 原始 Cu 对象
    strain_slab.set_cell(cell, scale_atoms=True, apply_constraint=False)
    # scale_Atoms=True must be set to True to ensure that ...
    # ...the atomic coordinates adapt to changes in the lattice matrix

    if is_opt == True:
        opt_strain_slab, opt_strain_energr = cal_LBFGC(strain_slab)
    else:
        opt_strain_slab, opt_strain_energr = strain_slab, False
    # strain = compare_atoms(ini_model,strain_slab)
    return (opt_strain_slab, opt_strain_energr)

In [6]:
def cal_BFGC(ini_model, potential=EMT, fmax=1e-6, steps=1000):
    # 执行动力学过程，默认的势函数是EMT，力收敛判断值1E-6，最大动力学步数1E3
    # 这个优化似乎不能放缩盒子
    ini_model.set_calculator(potential())  # setting the calculated potential
    # 创建 BFGS 实例
    dyn = BFGS(ini_model)
    # 进行能量最小化优化计算
    dyn.run(fmax, steps)

    # 输出优化后的结构信息和能量值
    opt_config = dyn.atoms  # initial model
    opt_energy = dyn.atoms.get_potential_energy()
    print("Limited-memory Broyden-Fletcher-Goldfarb-Shanno algorithm")
    return (opt_config, opt_energy)

In [7]:
def is_diagonal_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    return not any(
        i != j and matrix[i][j] != 0
        for i, j in itertools.product(range(rows), range(cols))
    )

In [8]:
def plot_model(model):
    def is_diagonal_matrix(matrix):
        rows = len(matrix)
        cols = len(matrix[0])
        return not any(
            i != j and abs(matrix[i][j]) > 0.1
            for i, j in itertools.product(range(rows), range(cols))
        )

    cell = model.get_cell()
    if is_diagonal_matrix(cell):
        # 绘制model构型的三视图
        fig, axs = plt.subplots(1, 3, dpi=600)
        fig.subplots_adjust(wspace=0.4, hspace=0)
        # 绘制第一个子图（俯视图）
        axs[0].set_aspect("equal")
        plot_atoms(
            model, axs[0], radii=0.9, rotation=("0x,0y,0z")
        )  # the "rotation" value is the  rotation angle of the axis
        axs[0].set_xlim(-1, cell[0, 0] + cell[1, 0] + 2)
        axs[0].set_ylim(-1, cell[1, 1] + 3)
        # axs[0].quiver(0.8, 0, 0.2, 0, color='r')
        axs[0].set_title("Top view", fontsize=10)
        # 绘制第二个子图（侧视图）
        axs[1].set_aspect("equal")
        plot_atoms(model, axs[1], radii=0.9, rotation=("-90x,0y,0z"))
        axs[1].set_xlim(-1, cell[0, 0] + cell[1, 0] + 4)
        axs[1].set_ylim(-1, cell[2, 2] + 3)
        axs[1].set_title("Front view", fontsize=10)
        # 绘制第三个子图（侧视图）
        axs[2].set_aspect("equal")
        plot_atoms(model, axs[2], radii=0.9, rotation=("-90x,90y,0z"))
        axs[2].set_xlim(-1, cell[1, 1] + 3)
        axs[2].set_ylim(-1, cell[2, 2] + 3)
        axs[2].set_title("Side view", fontsize=10)
        plt.show()
    else:
        cell = model.get_cell()
        # 绘制model构型的三视图
        fig, axs = plt.subplots(1, 3, dpi=600)
        fig.subplots_adjust(wspace=0.4, hspace=0)
        # 绘制第一个子图（俯视图）
        axs[0].set_aspect("equal")
        plot_atoms(
            model, axs[0], radii=0.9, rotation=("0x,0y,0z")
        )  # the "rotation" value is the  rotation angle of the axis
        axs[0].set_xlim(-1, cell[0, 0] + cell[1, 0] + 2)
        axs[0].set_ylim(-1, cell[1, 1] + 3)
        # axs[0].quiver(0.8, 0, 0.2, 0, color='r')
        axs[0].set_title("Top view", fontsize=10)
        # 绘制第二个子图（侧视图）
        axs[1].set_aspect("equal")
        plot_atoms(model, axs[1], radii=0.9, rotation=("-90x,0y,0z"))
        axs[1].set_xlim(-1, cell[0, 0] + cell[1, 0] + 4)
        axs[1].set_ylim(-1, cell[2, 2] + 3)
        axs[1].set_title("Front view", fontsize=10)
        # 绘制第个子图（侧视图）
        axs[2].set_aspect("equal")
        plot_atoms(model, axs[2], radii=0.9, rotation=("-90x,90y,0z"))
        axs[2].set_xlim(-5, cell[0, 0] + cell[1, 0])
        axs[2].set_ylim(-1, cell[2, 2] + 3)
        axs[2].set_title("Side view", fontsize=10)
        plt.show()

In [9]:
def plot_top(model_set, column=3):
    num = len(model_set)
    row = math.ceil(num / column)
    fig, axs = plt.subplots(row, column, dpi=600)
    fig.subplots_adjust(wspace=0.4, hspace=0)
    if row == 1 or column == 1:
        for i in range(num):
            model = model_set[i].copy()
            # 绘制第一个子图（俯视图）
            # print(adslab.cell)
            axs[i].set_aspect("equal")
            plot_atoms(
                model, axs[i], radii=0.8, rotation=("0x,0y,0z")
            )  # the "rotation" value is the  rotation angle of the axis
            axs[i].set_xlim(-1, model.cell[0, 0] + model.cell[1, 0] + 2)
            axs[i].set_ylim(-1, model.cell[1, 1] + 2)
    else:
        for i in range(num):
            a = math.floor(i / column)
            b = i % column
            model = model_set[i].copy()
            # 绘制第一个子图（俯视图）
            # print(adslab.cell)
            axs[a, b].set_aspect("equal")
            plot_atoms(
                model, axs[a, b], radii=0.8, rotation=("0x,0y,0z")
            )  # the "rotation" value is the  rotation angle of the axis
            axs[a, b].set_xlim(-1, model.cell[0, 0] + model.cell[1, 0] + 2)
            axs[a, b].set_ylim(-1, model.cell[1, 1] + 2)
    plt.show()

In [10]:
def lammps_relax(model):
    from lammps import lammps

    write(
        "bulk.lmp", model, format="lammps-data"
    )  # 输出bulk，文件名bulk.lmp，提供给lammps优化
    # lammps优化，输入是 bulk.lmp，输出是 opt_model.data，对三个方向进行放松，npt系综
    infile = "in.relax.in"
    lmp = lammps()
    lmp.file(infile)
    # 输出 opt_model.data
    atoms = lammpsdata.read_lammps_data("opt_model.data", style="atomic")
    new_model = model.copy()
    new_model.set_cell(atoms.get_cell(), scale_atoms=True)
    return new_model

In [11]:
def del_file(file_path):
    # 指定要删除的文件路径
    # 尝试删除文件
    if os.path.exists(file_path):
        try:
            os.remove(file_path)
            print(f"The file {file_path} was successfully deleted ")
        except OSError as e:
            print(f"Error deleting file {file_path}: {e}")
    else:
        print(f" {file_path} does not exist")

In [12]:
def poisson_disc_3d(box, num, isplot=False):
    import poisson_disc as pdis
    from mpl_toolkits.mplot3d import Axes3D

    box = np.array(box)
    dims3d = box[:, 1] - box[:, 0]
    cut = ((dims3d[0] * dims3d[1] * dims3d[2]) / 4 / num) ** (1 / 3)
    # print('cutoff=',cut)
    points3d = pdis.Bridson_sampling(dims3d, radius=cut)
    sort = list(range(len(points3d)))
    random.shuffle(sort)
    sort = sort[:num]
    points3d = points3d[sort]
    points3d = points3d + box[:, 0]
    if isplot == True:
        fig3d = plt.figure()
        # ax3d = Axes3D(fig3d)
        ax3d = Axes3D(fig3d, auto_add_to_figure=False)
        fig3d.add_axes(ax3d)
        ax3d.scatter(points3d[:, 0], points3d[:, 1], points3d[:, 2])
        plt.show()
    return points3d

In [13]:
def write_atoms(filename, atoms, format="vasp"):
    # 该函数的作用是修正 ase 的 write 函数，可以输出固定特定原子自由度的约束
    write(filename, atoms, format="vasp")
    if not any(
        isinstance(constraint, FixCartesian)
        for constraint in atoms.constraints
    ):
        return "write atoms only"
    cons = atoms.constraints
    # read part
    data = read_message(filename)  # 读取文件
    mbox, mnum, mdata, info = deal_message(data)  # 解析poscar
    for con in cons:
        if type(con) == FixCartesian:
            # deal part
            for i in con.a:
                string_list = info[i]
                result_list = [
                    string_list[i] if con.mask[i] else "F"
                    for i in range(len(con.mask))
                ]
                info[i] = result_list
    # write part
    newdata = joint(data, mbox, mnum, mdata, info)
    write_message(newdata, filename=filename)  # 输出poscar
    write(filename, read(filename), format="vasp")  # 读写检验
    return "write FixCartesian"


def out_poscar(input, path="New_Folder"):  # sourcery skip: extract-method
    # 如果文件夹已经存在，则删除该文件夹及其中的内容
    if os.path.exists(path):
        shutil.rmtree(path)
    # 创建新的文件夹
    os.makedirs(path)
    if isinstance(input, list):
        print("input is list")
        all_model = input
        for i in range(len(all_model)):
            atoms = all_model[i]
            filepath = os.path.join(".", path, str(int(i + 1e3)))
            os.makedirs(filepath)
            filename = os.path.join(filepath, "POSCAR")
            write_atoms(filename, atoms, format="vasp")
    elif isinstance(input, dict):
        print("input is dict")
        all_model = input
        name = all_model.keys()
        for key in name:
            atoms = all_model[key]
            filepath = os.path.join(".", path, key)
            os.makedirs(filepath)
            filename = os.path.join(filepath, "POSCAR")
            write_atoms(filename, atoms, format="vasp")
    elif isinstance(input, Atoms):
        print("input is Atoms")
        atoms = input
        filepath = path
        filename = os.path.join(filepath, "POSCAR")
        write_atoms(filename, atoms, format="vasp")
    else:
        raise ValueError("input must be a list or dict or Atoms")


def out_car_list(input, path="New_Folder"):
    # 这个函数中嵌套了out_poscar，所以实际输出是嵌套文件夹
    # 如果文件夹已经存在，则删除该文件夹及其中的内容
    if os.path.exists(path):
        shutil.rmtree(path)
    # 创建新的文件夹
    os.makedirs(path)
    if isinstance(input, list):
        for i in range(len(input)):
            filepath = os.path.join(".", path, str(int(i + 1e3)))
            os.makedirs(filepath)
            out_poscar(input[i], path=filepath)
    else:
        raise ValueError("input must be a list")


def get_dos_data(
    dos_folder, atom_num=6, orbital_list=list(range(9)), erange=[-11, 11]
):
    # 该函数可以获得指定文件夹的dos信息
    # 默认是POSCAR的第七个原子（atom_num=6），9个轨道，能量范围[-11,11]eV
    dos_data = standard.dos_atom_orbitals(
        atom_orbital_dict={atom_num: orbital_list},
        folder=dos_folder,
        erange=[-30, 30],
        total=False,
        save=False,
        figsize=[5, 3],
    )
    figure, axe = dos_data
    plt.close(figure)  # 关闭图像显示，不在命令行中展示
    x0 = axe.lines[0].get_data()[0]
    x = x0[(x0 > erange[0]) & (x0 <= erange[1])]
    dos_data = np.zeros((len(x), len(orbital_list) + 1))
    dos_data[:, 0] = x
    for i in range(len(orbital_list)):
        y = axe.lines[i].get_data()[-1]
        y = y[(x0 > -11.0) & (x0 <= 11.0)]
        dos_data[:, i + 1] = y
    return dos_data


def get_energy(OSZICAR_file, num=-1):
    # 该函数可以获得OSZICAR文件中的能量值，如果存在的话
    lines = []
    with open(OSZICAR_file, "r") as oszicar:
        # 逐行读取文件内容
        lines.extend(line for line in oszicar if " F= " in line)
    lastline = lines[num]
    if match := re.search(r"E0= ([-+]?\d*\.\d+E[+-]?\d+)", lastline):
        e0_value = match.group(1)
        print("E0 value:", num, "F  ", e0_value)
    else:
        print("No E0 value found.")
    e0_value = float(e0_value)
    return e0_value


def read_car(path, car="CONTCAR"):
    """
    可以从path中读取指定类型的文件，path可以是①计算文件夹，或者是②列表文件夹的父文件夹
    主要用于②，返回值是一个列表和一个字典，当子文件夹的名字是数字时，会返回升序排列好的列表和字典
    支持文件类型： car='CONTCAR' or 'POSCAR' or 'OSZICAR' or 'DOSCAR
    
    Args:
        path (str): Path can be a ① calculated folder, 
        or a ② parent folder of a list folder
        
        car (str, optional): car='CONTCAR' or 'POSCAR' or 'OSZICAR' or 'DOSCAR.
        Defaults to "CONTCAR".
        
    Returns:
    
        'CONTCAR' or 'POSCAR' return atoms of type ase.atom
        
        'OSZICAR' return Energy of type float.64
        
        'DOSCAR' return array of type np.array
    """

    if type(car) == list:
        num = car[1]
        car = car[0]
    else:
        num = -1
    all_list = []
    all_dict = {}
    subfolders = [
        f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))
    ]
    for subfolder in subfolders:
        folder_path = os.path.join(path, subfolder)
        filename = os.path.join(folder_path, car)
        if os.path.isfile(filename):
            if "OSZICAR" in car:
                atom = (
                    get_energy(filename, num=num)
                    if num != None
                    else get_energy(filename)
                )  # 可以指定要读取OSZICAR中的第几个离子步的能量
            elif "DOSCAR" in car:
                atom = get_dos_data(folder_path)
            else:
                atom = read(filename, format="vasp")
            all_list.append(atom)
            all_dict[subfolder] = atom
    if all(key.isdigit() for key in all_dict):  # 如果键的值都是数字，依据数字顺序调整列表和字典
        all_dict1 = collections.OrderedDict(
            sorted(all_dict.items(), key=lambda item: int(item[0]))
        )
        all_dict = dict(all_dict1)
        all_list = list(all_dict1.values())
    print('1st' ,all_dict.keys())
    return (all_list, all_dict)

def read_cars(source_folder, car="CONTCAR"):
    atoms_l = []
    atoms_d = {}
    atoms_d0 = {}  # 中间变量
    # 获取源文件夹的所有子文件夹
    subfolders = [
        f
        for f in os.listdir(source_folder)
        if os.path.isdir(os.path.join(source_folder, f))
    ]
    for subfolder in subfolders:
        
        subfolder_path = os.path.join(source_folder, subfolder)
        atom_l, atom_d = read_car(subfolder_path, car=car)
        atoms_l.extend(atom_l)
        atoms_d[subfolder] = atom_d  # 字典存字典
        atoms_d0[subfolder] = atom_l  # 字典存列表，用于排序列表
    if all(key.isdigit() for key in atoms_d):  # 如果键的值都是数字，依据数字顺序调整列表和字典
        atoms_d1 = collections.OrderedDict(
            sorted(atoms_d.items(), key=lambda item: int(item[0]))
        )
        atoms_d = dict(atoms_d1)  # 字典重排序

        atoms_d0 = collections.OrderedDict(
            sorted(atoms_d0.items(), key=lambda item: int(item[0]))
        )
        atoms_l = [
            value
            for key, value in sorted(
                atoms_d0.items(), key=lambda item: int(item[0])
            )
        ]
        # 此处的key有意义，保证了value不包含键值
        # 列表重排
    print("2nd", atoms_d.keys())
    return (atoms_l, atoms_d)


# 处理POSCAR函数模块
def read_message(filename):
    # 读取poscar
    message = []
    with open(filename, "r") as f:
        message.extend(line.strip() for line in f)
    return message


def deal_message(message):
    # 处理读取的文本
    mbox, _ = strs(message[2:5])
    mnum, _ = strs(message[6])
    mnum = [[int(element) for element in row] for row in mnum]
    num, info = strs(message[9 : sum(mnum[0]) + 9])
    mdata = num[:, 0:3]
    return mbox, mnum, mdata, info


def strs(mess):
    # deal_message 的子函数
    line = len(mess) if isinstance(mess, list) else 1
    num = []
    info = []
    for i in range(line):
        if isinstance(mess, list):
            number = re.split("[ ,]+", mess[i])
        else:
            number = re.split("[ ,]+", mess)
        if len(number) == 6 and isinstance(number[5], str):
            info.append(number[3:6])
            for m in range(3):
                number[m] = float(number[m])
        else:
            for m in range(len(number)):
                number[m] = float(number[m])
        num.append(number)
    num = np.array(num)
    return num, info


def write_message(message, filename="POSCAR"):
    # 写入poscar
    with open(filename, "w") as f:
        for line in message:
            f.write(line + "\n")
    print(f"write ------ {filename}")


def joint(data, mbox, mnum, mdata, info):
    # 将各部分组合
    newdata = data
    mbox = tostr(mbox)
    newdata[2:5] = mbox

    mnum = tostr(mnum)
    newdata[6] = mnum[0]

    data_info = np.concatenate((mdata, info), axis=1)
    data_info = tostr(data_info)
    newdata[9:] = data_info
    return newdata


def tostr(number_matrix):
    string_matrix = []
    for i in range(len(number_matrix)):
        number = number_matrix[i]
        newline = []
        for m in range(len(number)):
            if len(number) == 2:
                newline.append(str(int(number[m])))
            else:
                newline.append(str(number[m]))

        line = " ".join(newline)
        string_matrix.append(line)
    return string_matrix

In [14]:
def copy_contcar(
    rootdir,
    destdir=None,
    input_file="CONTCAR",
    output_put="POSCAR",
    func=shutil.copy2,
):
    # 该函数起到读取目录，构建对应目录，执行操作生成文件三个功能。
    # rootdir：已有的目录, destdir=None： 默认的生成的新目录, input_file='CONTCAR'：已有的文件,output_put='POSCAR'：生成的文件
    # func：源文件到新文件之间的操作，默认的操作是复制
    if destdir is None:
        destdir = f"{rootdir} _copy"
    if os.path.exists(destdir):
        shutil.rmtree(destdir)
    poscar_files = []
    for dirpath, _, filenames in os.walk(rootdir):
        # 构建对应的目标文件夹路径
        destpath = dirpath.replace(rootdir, destdir)
        if not os.path.exists(destpath):
            os.makedirs(destpath)
        for filename in filenames:
            if filename == input_file:
                # 构建新的文件名
                new_filename = output_put
                # 构建源文件路径和目标文件路径
                src_file = os.path.join(dirpath, filename)
                dest_file = os.path.join(destpath, new_filename)
                # 复制文件到目标路径
                func(src_file, dest_file)
                poscar_files.append(src_file)
                poscar_files = sorted(poscar_files)
    return poscar_files

In [15]:
from operator import itemgetter

def sort_z(data, ft=None):
    # 函数的目的是对z坐标排序，判断对应的坐标所在的层数，有一定的容错。
    if ft is None:
        ft = (max(data) - min(data)) / 1e2  # default fault-tolerant
    s = list(range(len(data)))  # 标记位置
    combined_arr = list(zip(data, s))
    sorted_arr = sorted(combined_arr, key=itemgetter(0))
    data = [x[0] for x in sorted_arr]
    s = [x[1] for x in sorted_arr]
    grouped_data = []
    current_group = []
    grouped_data_s = []
    current_group_s = []
    prev_value = None
    for i in range(len(s)):
        si = s[i]
        value = data[i]
        if prev_value is not None and abs(value - prev_value) >= ft:
            grouped_data.append(current_group)
            grouped_data_s.append(current_group_s)
            current_group = []  # 清空 重新记录
            current_group_s = []
        current_group.append(value)
        current_group_s.append(si)
        prev_value = value
    grouped_data.append(current_group)
    grouped_data_s.append(current_group_s)
    # 按照平均值从小到大排序
    grouped_data_s_with_mean = [
        (sum(group2) / len(group2), group1)
        for group1, group2 in zip(grouped_data_s, grouped_data)
    ]
    sorted_groups2 = sorted(grouped_data_s_with_mean, key=lambda x: x[0])
    # 排序后的类别
    sorted_categories2 = [group for _, group in sorted_groups2]

    tag = [0] * len(data)  # 初始的分类
    for i in range(len(sorted_categories2)):
        for j in sorted_categories2[i]:
            tag[j] = i
    return tag

In [16]:
def build_suface(
    bulk, vacuum_height=15.0, cell_z=None, relax_depth=2, iscala=False
):
    # 默认放松的原子层厚度为2
    if cell_z is not None:
        vacuum_height = cell_z - bulk.get_cell()[-1, -1]
        # 如果指定了期望的最终的cell的z方向高度，将采用期望高度
    # 创建真空层，生成表面
    adslab = bulk.copy()  # 无应变bulk构型
    if len(adslab.constraints):
        _extracted_from_build_suface_9(adslab, relax_depth)
    add_vacuum(adslab, vacuum_height)
    adslab.set_pbc(True)
    if iscala:
        adslab, adslab_e = cal_LBFGC(adslab)
    return adslab


# TODO Rename this here and in `build_suface`
def _extracted_from_build_suface_9(adslab, relax_depth):
    tags = np.zeros(len(adslab))
    layers = sort_z(adslab.get_positions()[:, 2])
    filtered_layers = [
        i for i in range(len(layers)) if layers[i] > max(layers) - relax_depth
    ]  #
    tags[filtered_layers] = 2  # 被吸附基底的表面层
    adslab.set_tags(tags)  # 将tags属性替换
    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    cons = FixAtoms(indices=[atom.index for atom in adslab if (atom.tag < 1)])
    adslab.set_constraint(cons)

In [17]:
def vac_ext(atom, vacuum_h=0.0, ads_layer=0):
    # 在原本的bulk上补充多层原子，符合构造规律，然后在z轴增加真空层
    # 相对于只允许复制的方式来扩展晶胞z方向，该命令运行可以很方便地指定扩展的层数
    slab = atom.copy()
    slab2 = slab.copy()
    pos = slab.get_positions()
    layers1 = sort_z(pos[:, 2])
    maxl = 2 + math.ceil(ads_layer / (max(layers1) + 1))
    slab2 = slab2.repeat((1, 1, maxl))
    pos2 = slab2.get_positions()
    layers2 = sort_z(pos2[:, 2])

    filtered_layers = [
        i for i in range(len(layers2)) if layers2[i] > ads_layer + max(layers1)
    ]  # 标记高于需求层之上的部分
    del slab2[filtered_layers]  # 大于需求层数的删去

    slab = slab2.copy()
    slab = build_suface(slab, cell_z=vacuum_h, iscala=False)

    layers3 = sort_z(slab.get_positions()[:, 2])
    filtered_layers2 = [
        i for i in range(len(layers3)) if layers3[i] > max(layers3) - 2
    ]  #
    tags = np.zeros(len(slab))
    tags[filtered_layers2] = 2  # 被吸附基底的表面层
    slab.set_tags(tags)  # 将tags属性替换
    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    cons0 = FixAtoms(indices=[atom.index for atom in slab if (atom.tag < 1)])
    cons1 = FixCartesian(filtered_layers2, mask=(1, 1, 0))
    slab.set_constraint([cons0, cons1])
    slab.set_pbc(True)
    # 打印更新后的结构
    return slab


def operate(src_file, dest_file):
    bulk = read(src_file)
    slab = vac_ext(bulk)
    write(dest_file, slab, format="vasp")

In [18]:
def generate_strain_matrix(
    num_strain,
    all_model_rotate,
    strain_box,
    random_select=0,
    uniform_strain=False,
    restart=False,
    dict_name="strain_matrix_dict.pkl",
    array_name="strain_matrix.npy",
):
    if random_select == 0:
        random_select = num_strain

    if restart:  # load
        try:
            with open(dict_name, "rb") as file:
                strain_matrix_dict = pickle.load(file)
            strain_matrix = np.load(array_name)
            return strain_matrix, strain_matrix_dict
        except FileNotFoundError:
            print("File not found. Generating new strain matrix.")

    if uniform_strain:
        strain = poisson_disc_3d(
            strain_box, num_strain, isplot=False
        )  # generate points
        strain_matrix_dict = {-1: strain}
    else:
        strain = poisson_disc_3d(
            strain_box, all_model_rotate * random_select, isplot=False
        )

    strain_matrix_dict = {}
    strain_matrix = np.zeros((all_model_rotate, random_select, 3))

    for i in range(all_model_rotate):
        if uniform_strain:
            random_numbers = random.sample(range(num_strain), random_select)
            random_numbers.sort()
        else:
            random_numbers = np.array(
                range(i * random_select, (i + 1) * random_select)
            )
        for j in range(len(random_numbers)):
            key = f"{str(i)}_{str(j)}"
            strain_matrix_dict[key] = strain[random_numbers[j]]
        strain_matrix[i] = strain[random_numbers]

    with open(dict_name, "wb") as file:  # save
        pickle.dump(strain_matrix_dict, file)
    np.save(array_name, strain_matrix)

    return strain_matrix, strain_matrix_dict


def test_dict_plot(data_dict):
    # 提取坐标数据
    if isinstance(data_dict, dict):
        print("Input is dict")
        data = list(data_dict.values())
        if isinstance(data[0], dict):
            data_dict = np.concatenate(data, axis=1)
        x_coords = [data_dict[key][0] for key in data_dict]
        y_coords = [data_dict[key][1] for key in data_dict]
        z_coords = [data_dict[key][2] for key in data_dict]
    elif isinstance(data_dict, np.ndarray):
        print("Input is array")
        x_coords = data_dict[:, 0]
        y_coords = data_dict[:, 1]
        z_coords = data_dict[:, 2]
    else:
        raise ValueError("Input not supported")

    # 创建画布和3D轴
    fig = plt.figure(figsize=(13, 13))
    ax = fig.add_subplot(111, projection="3d")
    # 给每个数据点的坐标计数
    coord_count = {}
    for i in range(len(x_coords)):
        coord = (x_coords[i], y_coords[i], z_coords[i])
        if coord in coord_count:
            coord_count[coord] += 5
        else:
            coord_count[coord] = 1
    # 绘制散点图，并根据坐标点数量设置点的大小
    for i in range(len(x_coords)):
        coord = (x_coords[i], y_coords[i], z_coords[i])
        size = 20 + coord_count[coord]  # 设置点的大小
        ax.scatter(
            x_coords[i], y_coords[i], z_coords[i], c="r", marker="o", s=size
        )
    # 设置坐标轴标签
    ax.set_xlabel("strain ε_x", fontsize=12)
    ax.set_ylabel("strain ε_y", fontsize=12)
    ax.set_zlabel("strain 2ε_xy", fontsize=12)  # 显示图形
    plt.show()

In [19]:
def set_cons0(model):
    """
    此约束的效果是：固定表面两层以下的原子
    """
    slab = model.copy()
    layers = sort_z(slab.get_positions()[:, 2])
    tags = np.zeros(len(slab))

    filtered_layers2 = [
        i for i in range(len(layers)) if layers[i] > max(layers) - 2
    ]  #
    tags[filtered_layers2] = 1  # 被吸附基底的次表面和表面

    filtered_layers1 = [
        i for i in range(len(layers)) if layers[i] > max(layers) - 1
    ]  #
    tags[filtered_layers1] = 2  # 被吸附基底的表面层

    slab.set_tags(tags)  # 将tags属性替换
    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    cons0 = FixAtoms(indices=[atom.index for atom in slab if (atom.tag < 1)])
    cons1 = FixCartesian(filtered_layers2, mask=(1, 1, 0))
    slab.set_constraint([cons0, cons1])
    slab.set_pbc(True)
    return slab


def set_cons1(model):
    """
    此约束的效果是：固定表面一层以下 和 左下角 的原子
    """
    f_pos = model.get_scaled_positions()
    slab = model.copy()
    layers = sort_z(slab.get_positions()[:, 2])
    tags = np.zeros(len(slab))

    filtered_layers2 = [
        i for i in range(len(layers)) if layers[i] > max(layers) - 2
    ]  #
    tags[filtered_layers2] = 1  # 被吸附基底的次表面和表面

    filtered_layers1 = [
        i for i in range(len(layers)) if layers[i] > max(layers) - 1
    ]  #
    tags[filtered_layers1] = 2  # 被吸附基底的表面层

    filtered_layers0 = [
        i
        for i in range(len(model))
        if tags[i] == 2 and f_pos[i, 0] + f_pos[i, 1] < 0.5
    ]  #
    tags[filtered_layers0] = -1  # 左下角的三个原子

    slab.set_tags(tags)  # 将tags属性替换
    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    cons0 = FixAtoms(indices=[atom.index for atom in slab if (atom.tag < 2)])
    # cons1 = FixCartesian(filtered_layers2,mask=(1, 1, 0))
    slab.set_constraint([cons0])
    slab.set_pbc(True)
    return slab


def set_cons2(model):
    """
    此约束的效果是：固定所有Cu原子，而氧原子固定x和y方向
    """

    slab = model.copy()
    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    atomic_numbers = slab.get_atomic_numbers()
    cons0 = FixAtoms(
        indices=[
            slab[i].index for i in range(len(slab)) if atomic_numbers[i] != 8
        ]
    )
    filtered_layers1 = [
        i for i in range(len(atomic_numbers)) if atomic_numbers[i] == 8
    ]  #
    cons1 = FixCartesian(filtered_layers1, mask=(1, 1, 0))
    slab.set_constraint([cons0, cons1])
    slab.set_pbc(True)
    return slab


def set_cons3(model):
    """
    此约束的效果是：固定表面一层以下 和 左下角 的原子，而氧原子固定x和y方向
    """
    model1 = model.copy()
    model0 = model.copy()
    atomic_numbers = model0.get_atomic_numbers()
    filtered_layers1 = [
        i for i in range(len(atomic_numbers)) if atomic_numbers[i] == 8
    ]  #
    del model0[filtered_layers1]  # 先暂时删除氧原子

    model0 = set_cons1(model0)
    cons0 = model0.constraints[0]
    cons1 = FixCartesian([filtered_layers1], mask=(1, 1, 0))
    model1.set_constraint([cons0, cons1])  # 在之前约束的基础上又约束了两个

    # Fixed atoms are prevented from moving during a structure relaxation. We fix all slab atoms beneath the surface
    model1.set_pbc(True)
    return model1

# 1. coed

In [20]:
import random

def generate_random_number():
    # 生成x坐标在一三象限的随机数
    x = random.random() * 100  # 这里假设范围是0到100
    y = random.random() * 100  # 这里假设范围是0到100
    return x, y

# 测试生成随机数
random_number = generate_random_number()
print("随机数:", random_number)


随机数: (67.41127007303079, 77.21295406248012)


In [21]:
# 计划生成三组随机数 分别为 单轴拉伸和压缩 沿x和y方向两组 和等轴拉伸压缩
# 直接采用等间隔生成数据

In [22]:
array_name = '../code/strain_matrix.npy'
dict_name = '../code/strain_matrix_dict.pkl'
restart = True
if restart:  # load
    try:
        with open(dict_name, "rb") as file:
            strain_matrix_dict = pickle.load(file)
        strain_matrix = np.load(array_name)

    except FileNotFoundError:
        print("File not found. Generating new strain matrix.")

In [23]:
strain_matrix_dict.keys()

dict_keys(['0_0', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '0_10', '0_11', '0_12', '0_13', '0_14', '0_15', '0_16', '0_17', '0_18', '0_19', '0_20', '0_21', '1_0', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '1_11', '1_12', '1_13', '1_14', '1_15', '1_16', '1_17', '1_18', '1_19', '1_20', '1_21', '2_0', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '2_9', '2_10', '2_11', '2_12', '2_13', '2_14', '2_15', '2_16', '2_17', '2_18', '2_19', '2_20', '2_21', '3_0', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '3_9', '3_10', '3_11', '3_12', '3_13', '3_14', '3_15', '3_16', '3_17', '3_18', '3_19', '3_20', '3_21', '4_0', '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '4_7', '4_8', '4_9', '4_10', '4_11', '4_12', '4_13', '4_14', '4_15', '4_16', '4_17', '4_18', '4_19', '4_20', '4_21', '5_0', '5_1', '5_2', '5_3', '5_4', '5_5', '5_6', '5_7', '5_8', '5_9', '5_10', '5_11', '5_12', '5_13', '5_14', '5_15', '5_16', '5_17', '5_18', '5_19', '5_20', '

In [24]:
strain_matrix_dict

{'0_0': array([ 0.01896039, -0.0068634 , -0.0097765 ]),
 '0_1': array([-0.00205553,  0.00474957, -0.00268788]),
 '0_2': array([-0.01410769,  0.00386953,  0.00773346]),
 '0_3': array([-0.01282268, -0.01218621, -0.0061792 ]),
 '0_4': array([-0.01293477,  0.01623199, -0.00888301]),
 '0_5': array([-0.01345327,  0.01772644, -0.00638697]),
 '0_6': array([ 0.00309671, -0.00359557, -0.0080248 ]),
 '0_7': array([-0.01833234, -0.0195452 ,  0.0075357 ]),
 '0_8': array([ 0.01468731,  0.0057494 , -0.00622603]),
 '0_9': array([-0.0043027 , -0.01906025, -0.00659573]),
 '0_10': array([-1.38881067e-02, -1.13122910e-05, -4.87604201e-03]),
 '0_11': array([0.00722727, 0.00584536, 0.00893645]),
 '0_12': array([-0.00569807,  0.01838575,  0.00886266]),
 '0_13': array([-0.01980875, -0.01172144,  0.00376595]),
 '0_14': array([ 0.01926582, -0.01267439, -0.00873199]),
 '0_15': array([ 0.00673404, -0.00802081,  0.00729699]),
 '0_16': array([ 0.01414198, -0.01968552,  0.0066375 ]),
 '0_17': array([-0.01589312, -0.

In [25]:
strain_matrix.shape

(27, 22, 3)

In [ ]:
# 创建x和y同正负，z在小范围波动的空间

In [206]:
strain_max = 0.03
num_strain = 7 # 包含原点，实际上输出没有原点
all_roate_model = 6
strain_matrix = np.zeros((all_roate_model,num_strain*3,3))

strain = []
for i in range(num_strain):
    ii = i/(num_strain-1)*2-1
    # if ii != 0:
    print(ii*strain_max)
    strain.append(ii*strain_max)
        
strain_matrix = np.zeros((all_roate_model,len(strain)*3,3))

for j in range(all_roate_model):
    for i in range(len(strain)):
        strain_matrix[j,i,0] = strain[i]+0.01*(j/all_roate_model)

    for i in range(len(strain)):
        strain_matrix[j,i+len(strain),1] = strain[i]+0.01*(j/all_roate_model)

    for i in range(len(strain)):
        strain_matrix[j,i+len(strain)*2,:2] = strain[i]+0.01*(j/all_roate_model)


-0.03
-0.02
-0.01
0.0
0.009999999999999997
0.02
0.03


In [207]:
strain_matrix

array([[[-0.03      ,  0.        ,  0.        ],
        [-0.02      ,  0.        ,  0.        ],
        [-0.01      ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.01      ,  0.        ,  0.        ],
        [ 0.02      ,  0.        ,  0.        ],
        [ 0.03      ,  0.        ,  0.        ],
        [ 0.        , -0.03      ,  0.        ],
        [ 0.        , -0.02      ,  0.        ],
        [ 0.        , -0.01      ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.01      ,  0.        ],
        [ 0.        ,  0.02      ,  0.        ],
        [ 0.        ,  0.03      ,  0.        ],
        [-0.03      , -0.03      ,  0.        ],
        [-0.02      , -0.02      ,  0.        ],
        [-0.01      , -0.01      ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.01      ,  0.01      ,  0.        ],
        [ 0.02      ,  0.02      ,  0.        ],
        [ 0.03      

In [208]:
strain_matrix_dict = {}
for i in range(strain_matrix.shape[0]):
    for j in range(strain_matrix.shape[1]):
        key = f"{str(i)}_{str(j)}"
        strain_matrix_dict[key] = strain_matrix[i,j,:]
        

In [209]:
strain_matrix_dict

{'0_0': array([-0.03,  0.  ,  0.  ]),
 '0_1': array([-0.02,  0.  ,  0.  ]),
 '0_2': array([-0.01,  0.  ,  0.  ]),
 '0_3': array([0., 0., 0.]),
 '0_4': array([0.01, 0.  , 0.  ]),
 '0_5': array([0.02, 0.  , 0.  ]),
 '0_6': array([0.03, 0.  , 0.  ]),
 '0_7': array([ 0.  , -0.03,  0.  ]),
 '0_8': array([ 0.  , -0.02,  0.  ]),
 '0_9': array([ 0.  , -0.01,  0.  ]),
 '0_10': array([0., 0., 0.]),
 '0_11': array([0.  , 0.01, 0.  ]),
 '0_12': array([0.  , 0.02, 0.  ]),
 '0_13': array([0.  , 0.03, 0.  ]),
 '0_14': array([-0.03, -0.03,  0.  ]),
 '0_15': array([-0.02, -0.02,  0.  ]),
 '0_16': array([-0.01, -0.01,  0.  ]),
 '0_17': array([0., 0., 0.]),
 '0_18': array([0.01, 0.01, 0.  ]),
 '0_19': array([0.02, 0.02, 0.  ]),
 '0_20': array([0.03, 0.03, 0.  ]),
 '1_0': array([-0.02833333,  0.        ,  0.        ]),
 '1_1': array([-0.01833333,  0.        ,  0.        ]),
 '1_2': array([-0.00833333,  0.        ,  0.        ]),
 '1_3': array([0.00166667, 0.        , 0.        ]),
 '1_4': array([0.0116666

In [210]:
dict_name  = './vasp_cala/tension_compression/strain_matrix_dict.pkl'
array_name = './vasp_cala/tension_compression/strain_matrix.npy'

with open(dict_name, "wb") as file:  # save
    pickle.dump(strain_matrix_dict, file)
np.save(array_name, strain_matrix)